<table>
    <tr>
        <td><img src="./img/Macc.png" width="auto"/></td>
        <td>
            <table><tr>
            <h1 style="color:blue;text-align:center">Lógica para Ciencias de la Computación</h1></td>
            </tr></table>   
        <td>&nbsp;</td>
        <td>
            <table><tr>
            <tp><p style="font-size:150%;text-align:center">Taller</p></tp>
            <tp><p style="font-size:150%;text-align:center">Implementación de Tableaux</p></tp>
            </tr></table>
        </td>
    </tr>
</table>

---

# Objetivos <a class="anchor" id="inicio"></a>

El objetivo de este notebook es implementar el algoritmo de construcción de tableros semánticos (tableaux). El método de los tableaux es un SATsolver, el cual recibe una fórmula y devuelve una interpretación de la misma, si es que existe, o informa que la fórmula es insatisfacible. Veremos diferentes maneras de implementar este método, dependiendo de distintas maneras de expandir el árbol del tableaux.


# Secciones

1. [Clasificación de fórmulas para tableaux.](#clas)
2. [Expansión primero en anchura.](#anchura)
3. [Expansión primero en profundidad.](#prof)
4. [Recorrido backtracking.](#backt)
5. [Eficiencia de los algoritmos (opcional).](#efi)

# Clasificación de fórmulas para tableaux <a class="anchor" id="clas"></a>

([Volver al inicio](#inicio))

Recordemos el siguiente diagrama de flujo para la implementación de un tableaux $\tau$ para una fórmula $A$:

<img src="./img/flow.png" width="600"/>

Hay tres elementos centrales en este algoritmo:

* Clasificación de una fórmula como $\alpha$ o $\beta$.
* Implementación de los nodos.
* Selección del próximo nodo a expandir.

En esta sección veremos la clasificación de fórmulas y la implementación de los nodos. En las siguientes secciones veremos tres maneras distintas de seleccionar el próximo nodo a expandir.

### Clasificación de una fórmula como $\alpha$ o $\beta$

Para realizar la implementación de los tableaux, el orden más apropiado es primero implementar la clasificación de fórmulas. Hemos visto que toda fórmula o es un literal, o puede clasificarse como una fórmula de tipo $\alpha$ o $\beta$. Para referencia, repetimos las clasificaciones en la siguiente figura:

<img src="./img/clasif.png" width="200px"/>

En el siguiente código hemos implementado la clasificación para dos casos: literales y fórmulas 1$\alpha$:

In [ ]:
from ProyectoLogica.Logica import *

In [2]:
def clasifica_para_tableaux(self):
    if type(self) == Letra:
        return None, 'literal'          # Literal positivo
    if type(self) == Negacion:
        if type(self.subf) == Letra:
            return None, 'literal'      # Literal negativo
        if type(self.subf) == Negacion:
            return 1, 'alfa'            # Fórmula 1 alfa
        if type(self.subf) == Binario:
            if self.subf.conectivo == 'Y':
                return 1, 'beta'
            if self.subf.conectivo == 'O':
                return 3, 'alfa'
            if self.subf.conectivo == '>':
                return 4, 'alfa'
            if self.subf.conectivo == '=':
                return 5, 'beta'
    if type(self) == Binario:
        if self.conectivo == 'Y':
            return 2, 'alfa'
        if self.conectivo == 'O':
            return 2, 'beta'
        if self.conectivo == '>':
            return 3, 'beta'
        if self.conectivo == '=':
            return 4, 'beta'

setattr(Formula,"clasifica_para_tableaux",clasifica_para_tableaux)

**Ejercicio 1:**

Complete la anterior función `clasifica_para_tableaux` para clasificar el resto de fórmulas de acuerdo a su tipo. Pruebe su implementación con las siguientes fórmulas:

**Nota:** La respuesta debe ser:

```
(1, 'alfa')
(2, 'alfa')
(3, 'alfa')
(4, 'alfa')
(1, 'beta')
(2, 'beta')
(3, 'beta')
```

In [3]:
f = '--(-(pOq)Y-(r>s))'  # 1, alfa
A = inorder_to_tree(f)
print(A.clasifica_para_tableaux())

f = '(-(pOq)Y-(r>s))'  # 2, alfa
A = inorder_to_tree(f)
print(A.clasifica_para_tableaux())

f = '-(pOq)'  # 3, alfa
A = inorder_to_tree(f)
print(A.clasifica_para_tableaux())

f = '-(r>s)' # 4, alfa
A = inorder_to_tree(f)
print(A.clasifica_para_tableaux())

f = '-(pYq)' # 1, beta
A = inorder_to_tree(f)
print(A.clasifica_para_tableaux())

f = '(-(pYq)O(r>s))' # 2, beta
A = inorder_to_tree(f)
print(A.clasifica_para_tableaux())

f = '-g' # None, literal
A = inorder_to_tree(f)
print(A.clasifica_para_tableaux())

(1, 'alfa')
(2, 'alfa')
(3, 'alfa')
(4, 'alfa')
(1, 'beta')
(2, 'beta')
(None, 'literal')


---

### Implementación de los nodos

El tableaux que vamos a implementar está basado en una estructura de datos conocida como árbol, el cual está basado en nodos. Los nodos son de la clase `nodos_tableaux` que se encuentra en la librería `Logica`. La descripción de la clase es la siguiente:

**Atributos:**

* `alfa`: una lista con las fórmulas tipo $\alpha$, representadas como una 4-tupla que incluye la fórmula como árbol, como cadena, el número de la regla y la cadena 'alfa'.
* `beta`: una lista con las fórmulas tipo $\beta$, representadas como una 4-tupla que incluye la fórmula como árbol, como cadena, el número de la regla y la cadena 'beta'.
* `literales`: una lista con los literales, representados como una 4-tupla que incluye la fórmula como árbol, como cadena, None y la cadena 'literal'.

**Métodos:**

* `tiene_lit_comp()`: retorna `True` si self.`literales` tiene un par complementario de literales.
* `es_hoja()`: retorna 'cerrada' si self.`literales` tiene un par complementario de literales; 'abierta' si self.`literales` NO tiene un par complementario de literales y no tiene reglas ni alfa ni beta; None en otro caso.
* `interp()`: retorna un diccionario que hace veradero a todos los literales en self.`literales`.
* `expandir()`: retorna una lista con un único `nodo_tableaux`, el cual es el resultado de aplicar la primera regla $\alpha$. Si no hay reglas $\alpha$, retorna una lista con dos `nodo_tableaux` que son el resultado de aplicar la primera regla $\beta$. Si no hay reglas $\beta$, retorna una lista vacía.

Para ilustrar esta clase, instanciemos un objeto a partir de una fórmula $\alpha$, una $\beta$ y un literal:

In [4]:
print('Nodo n:')
forms = ['-(p>q)', '(pOq)', '-p']
forms = [inorder_to_tree(f) for f in forms]
n = nodos_tableaux(forms)
print(n)
print('')
print('¿Tiene literales complementarios?', n.tiene_lit_comp())
print('¿Es una hoja?', n.es_hoja())
print('Interpretación para literales:', n.interp())

Nodo n:
Alfas:['-(p>q)']
Betas:['(pOq)']
Literales:['-p']

¿Tiene literales complementarios? False
¿Es una hoja? None
Interpretación para literales: {'p': False}


El método `expandir` tomará la primera (y única en este caso) fórmula $\alpha$ y devolvera una lista [`n1`] donde `n1` es el nodo resultado de aplicar la regla:

In [5]:
hijos = n.expandir()
hijos

In [6]:
for hijo in hijos:
    print(hijo)

Alfas:[]
Betas:['(pOq)']
Literales:['-p', 'p', '-q']


**Ejercicio 2:**

Explanda el hijo de `n` y verifique si los nodos resultado tienen pares complementarios de literales.

In [7]:
for hijo in hijos:
    print(hijo)
    print(hijo.tiene_lit_comp())

Alfas:[]
Betas:['(pOq)']
Literales:['-p', 'p', '-q']
True


---

**Ejercicio 3:**

Expanda manualmente (o mediante un while) el siguiente nodo hasta obtener solo una lista de literales y devuelva la interpretación que los hace verdaderos. Observe que todas sus subformulas son de tipo $\alpha$:

In [8]:
forms = ['--(-(pOq)Y-(r>s))']
forms = [inorder_to_tree(f) for f in forms]
n = nodos_tableaux(forms)
hijos = n.expandir()
hijos2 = hijos[0].expandir()
for hijo in hijos2:
    print(hijo)

Alfas:['-(pOq)', '-(r>s)']
Betas:[]
Literales:[]


---

# Expansión primero en anchura <a class="anchor" id="anchura"></a>

([Volver al inicio](#inicio))

Ya tenemos los elementos necesarios para la generación del tableaux, pero necesitamos especificar una manera de ir expandiendo el árbol. Además de la opción de expandirlo de manera aleatoria -- seleccionando aleatoriamente un nodo que no sea una hoja -- tenemos tres opciones. En esta sección examinaremos la expansión primero en anchura. El pseudocódigo del algoritmo es el siguiente:

<img src="./img/anchura.png" width="350px"/>

La implementación en Python es la siguiente:

In [9]:
def primero_anchura(nodo):
    estado = nodo
    res = estado.es_hoja()
    if res == 'cerrada':
        return None
    elif res == 'abierta':
        return estado.interp()
    frontera = [estado]
    while len(frontera) > 0:
        estado = frontera.pop(0) 
        hijos = estado.expandir()
        for a in hijos:
            res = a.es_hoja()
            if res == 'abierta':
                return a.interp()
            elif res == None:
                frontera.append(a)
    return None

Podemos usar esta función para expandir el nodo del ejercicio 3:

In [10]:
forms = ['--(-(pOq)Y-(r>s))']
forms = [inorder_to_tree(f) for f in forms]
n = nodos_tableaux(forms)
I = primero_anchura(n)
print("Resultado:", I)

Resultado: {'p': False, 'q': False, 'r': True, 's': False}


**Ejercicio 4:**

Expanda el nodo correspondiente a la siguiente fórmula:

'(((pYq)O(rOq))Y(-qO-p))'

Debe obtener la siguiente interpretación:

Resultado: {'q': False, 'r': True}

In [11]:
forms = ['(((pYq)O(rOq))Y(-qO-p))']
forms = [inorder_to_tree(f) for f in forms]
n = nodos_tableaux(forms)
I = primero_anchura(n)
print("Resultado:", I)

Resultado: {'q': False, 'r': True}


---

# Expansión primero en profundidad <a class="anchor" id="prof"></a>

([Volver al inicio](#inicio))

Además de poder expandir primero en anchura, podemos expandir primero en profundidad. El pseudocódigo de este algoritmo es el siguiente:

<img src="./img/profundidad.png" width="350px"/>

In [12]:
def primero_profundidad(nodo):
    estado = nodo
    res = estado.es_hoja()
    if res == 'cerrada':
        return None
    elif res == 'abierta':
        return estado.interp()
    frontera = [estado]
    while len(frontera) > 0:
        estado = frontera.pop() 
        hijos = estado.expandir()
        for a in hijos:
            res = a.es_hoja()
            if res == 'abierta':
                return a.interp()
            elif res == None:
                frontera.append(a)
    return None

**Ejercicio 5:**

Implemente la función `primero_profundidad` y expanda los nodos para las siguientes fórmulas:

* '--(-(pOq)Y-(r>s))'
* '(((pYq)O(rOq))Y(-qO-p))'

Las soluciones deben ser las siguientes:

* Resultado: {'p': False, 'q': False, 'r': True, 's': False}
* Resultado: {'p': False, 'r': True}

In [13]:
forms = ['--(-(pOq)Y-(r>s))']
forms = [inorder_to_tree(f) for f in forms]
n = nodos_tableaux(forms)
I = primero_profundidad(n)
print("Resultado:", I)

Resultado: {'p': False, 'q': False, 'r': True, 's': False}


In [14]:
forms = ['(((pYq)O(rOq))Y(-qO-p))']
forms = [inorder_to_tree(f) for f in forms]
n = nodos_tableaux(forms)
I = primero_profundidad(n)
print("Resultado:", I)

Resultado: {'p': False, 'r': True}


---

# Recorrido con backtracking <a class="anchor" id="backt"></a>

([Volver al inicio](#inicio))

Podemos hacer también un recorrido por el árbol haciendo una expansión recursiva que implementa el backtracking, como en el siguiente pseudocódigo:

<img src="./img/backtracking.png" width="350px"/>

In [15]:
def backtracking(nodo):
    estado = nodo
    res = estado.es_hoja()
    if res == 'cerrada':
        return None
    elif res == 'abierta':
        return estado.interp()
    for hijo in estado.expandir():
        resultado = backtracking(hijo)
        if resultado != None:
            return resultado
    return None

**Ejercicio 6:**

Implemente la función `backtracking` y expanda los nodos para las siguientes fórmulas:

* '--(-(pOq)Y-(r>s))'
* '(((pYq)O(rOq))Y(-qO-p))'

Las soluciones deben ser las siguientes:

* Resultado: {'p': False, 'q': False, 'r': True, 's': False}
* Resultado: {'q': False, 'r': True}

In [16]:
forms = ['--(-(pOq)Y-(r>s))']
forms = [inorder_to_tree(f) for f in forms]
n = nodos_tableaux(forms)
I = backtracking(n)
print("Resultado:", I)

Resultado: {'p': False, 'q': False, 'r': True, 's': False}


In [17]:
forms = ['(((pYq)O(rOq))Y(-qO-p))']
forms = [inorder_to_tree(f) for f in forms]
n = nodos_tableaux(forms)
I = backtracking(n)
print("Resultado:", I)

Resultado: {'q': False, 'r': True}


In [18]:
import numpy as np

class Descriptor :

    '''
    Codifica un descriptor de N argumentos mediante un solo caracter
    Input:  args_lista, lista con el total de opciones para cada
                     argumento del descriptor
            chrInit, entero que determina el comienzo de la codificación chr()
    Output: str de longitud 1
    '''

    def __init__ (self,args_lista,chrInit=256) -> None:
        self.args_lista = args_lista
        assert(len(args_lista) > 0), "Debe haber por lo menos un argumento"
        self.chrInit = chrInit
        self.rango = [chrInit, chrInit + np.prod(self.args_lista)]

    def check_lista_valores(self,lista_valores: List[int]) -> None:
        for i, v in enumerate(lista_valores) :
            assert(v >= 0), "Valores deben ser no negativos"
            assert(v < self.args_lista[i]), f"Valor debe ser menor o igual a {self.args_lista[i]}"

    def codifica(self,lista_valores: List[int]) -> int:
        self.check_lista_valores(lista_valores)
        cod = lista_valores[0]
        n_columnas = 1
        for i in range(0, len(lista_valores) - 1) :
            n_columnas = n_columnas * self.args_lista[i]
            cod = n_columnas * lista_valores[i+1] + cod
        return cod

    def decodifica(self,n: int) -> int:
        decods = []
        if len(self.args_lista) > 1:
            for i in range(0, len(self.args_lista) - 1) :
                n_columnas = np.prod(self.args_lista[:-(i+1)])
                decods.insert(0, int(n / n_columnas))
                n = n % n_columnas
        decods.insert(0, n % self.args_lista[0])
        return decods

    def ravel(self,lista_valores: List[int]) -> chr:
        codigo = self.codifica(lista_valores)
        return chr(self.chrInit+codigo)

    def unravel(self,codigo: chr) -> int:
        n = ord(codigo)-self.chrInit
        return self.decodifica(n)
    
    def escribir(self, literal: chr) -> str:
        if '-' in literal:
            atomo = literal[1:]
            neg = ' no'
        else:
            atomo = literal
            neg = ''
        n, r, c  = self.unravel(atomo)
        return f"PREDICADO({n, r, c})"        
    
    

def visualizar_formula(A: Formula, D: Descriptor) -> str:
    '''
    Visualiza una fórmula A (como string en notación inorder) usando el descriptor D
    '''
    vis = []
    for c in A:
        if c == '-':
            vis.append(' no ')
        elif c in ['(', ')']:
            vis.append(c)
        elif c in ['>', 'Y', 'O']:
            vis.append(' ' + c + ' ')
        elif c == '=':
            vis.append(' sii ')
        else:
            try:
                vis.append(D.escribir(c))
            except:
                raise("¡Caracter inválido!")
    return ''.join(vis)

# Quiz

In [19]:
Nx = 4
Ny = 4
Nn = Nx * Ny
Nf = 3
X = list(range(Nx))
Y = list(range(Ny))
F = list(range(Nf))
nombres_F = {0: 'tr', 1: 'cu', 2: 'ci'}

casillas = list(range(Nn))
FichaEn = Descriptor([Nx, Ny, Nf])

In [20]:
print("Cantidad de átomos FichaEn:", FichaEn.rango[1] - FichaEn.rango[0])
print("Caracteres correspondientes a los átomos OenCasilla:\n")
for x in range(Nx):
    for y in range(Ny):
        for f in range(Nf):
            atomo = FichaEn.ravel([x,y,f])
            print(f"Que la figura {nombres_F[f]} está en la casilla ({X[x]},{Y[y]}) es el átomo {atomo}")
    print("")

Cantidad de átomos FichaEn: 48
Caracteres correspondientes a los átomos OenCasilla:

Que la figura tr está en la casilla (0,0) es el átomo Ā
Que la figura cu está en la casilla (0,0) es el átomo Đ
Que la figura ci está en la casilla (0,0) es el átomo Ġ
Que la figura tr está en la casilla (0,1) es el átomo Ą
Que la figura cu está en la casilla (0,1) es el átomo Ĕ
Que la figura ci está en la casilla (0,1) es el átomo Ĥ
Que la figura tr está en la casilla (0,2) es el átomo Ĉ
Que la figura cu está en la casilla (0,2) es el átomo Ę
Que la figura ci está en la casilla (0,2) es el átomo Ĩ
Que la figura tr está en la casilla (0,3) es el átomo Č
Que la figura cu está en la casilla (0,3) es el átomo Ĝ
Que la figura ci está en la casilla (0,3) es el átomo Ĭ

Que la figura tr está en la casilla (1,0) es el átomo ā
Que la figura cu está en la casilla (1,0) es el átomo đ
Que la figura ci está en la casilla (1,0) es el átomo ġ
Que la figura tr está en la casilla (1,1) es el átomo ą
Que la figura cu e

## Restricción 4
No puede repetirse la misma figura en una fila.

$$\bigwedge_{x \in X}\bigwedge_{y \in Y}\bigwedge_{f \in F} (FichaEn(x, y, f) \rightarrow \neg ( \bigvee_{c \in X - \{x\}} FichaEn(c,y,f)))$$

In [21]:
print("=== Implementación de Restricción 4 ===")

inicial3 = True
final = ''
for x in X:
    for y in Y:
        inicial2 = True
        formula2 = ''
        for f in F:  # ahora F = [0,1,2]
            formula1 = ''
            inicial = True
            otras_columnas = [c for c in X if c != x]
            for c in otras_columnas:
                if inicial:
                    formula1 = FichaEn.ravel([c, y, f])
                    inicial = False
                else:
                    formula1 = "(" + formula1 + "O" + FichaEn.ravel([c, y, f]) + ")"
            # implicación con negación
            f1 = "(" + FichaEn.ravel([x, y, f]) + ">-" + formula1 + ")"
            if inicial2:
                formula2 = f1
                inicial2 = False
            else:
                formula2 = "(" + formula2 + "Y" + f1 + ")"
        if inicial3:
            final = formula2
            inicial3 = False
        else:
            final = "(" + final + "Y" + formula2 + ")"

print(visualizar_formula(final, FichaEn))


=== Implementación de Restricción 4 ===
((((((((((((((((((PREDICADO((np.int64(0), 0, 0)) >  no ((PREDICADO((np.int64(1), 0, 0)) O PREDICADO((np.int64(2), 0, 0))) O PREDICADO((np.int64(3), 0, 0)))) Y (PREDICADO((np.int64(0), 0, 1)) >  no ((PREDICADO((np.int64(1), 0, 1)) O PREDICADO((np.int64(2), 0, 1))) O PREDICADO((np.int64(3), 0, 1))))) Y (PREDICADO((np.int64(0), 0, 2)) >  no ((PREDICADO((np.int64(1), 0, 2)) O PREDICADO((np.int64(2), 0, 2))) O PREDICADO((np.int64(3), 0, 2))))) Y (((PREDICADO((np.int64(0), 1, 0)) >  no ((PREDICADO((np.int64(1), 1, 0)) O PREDICADO((np.int64(2), 1, 0))) O PREDICADO((np.int64(3), 1, 0)))) Y (PREDICADO((np.int64(0), 1, 1)) >  no ((PREDICADO((np.int64(1), 1, 1)) O PREDICADO((np.int64(2), 1, 1))) O PREDICADO((np.int64(3), 1, 1))))) Y (PREDICADO((np.int64(0), 1, 2)) >  no ((PREDICADO((np.int64(1), 1, 2)) O PREDICADO((np.int64(2), 1, 2))) O PREDICADO((np.int64(3), 1, 2)))))) Y (((PREDICADO((np.int64(0), 2, 0)) >  no ((PREDICADO((np.int64(1), 2, 0)) O PREDICADO

In [22]:
print(final)

((((((((((((((((((Ā>-((āOĂ)Oă))Y(Đ>-((đOĒ)Oē)))Y(Ġ>-((ġOĢ)Oģ)))Y(((Ą>-((ąOĆ)Oć))Y(Ĕ>-((ĕOĖ)Oė)))Y(Ĥ>-((ĥOĦ)Oħ))))Y(((Ĉ>-((ĉOĊ)Oċ))Y(Ę>-((ęOĚ)Oě)))Y(Ĩ>-((ĩOĪ)Oī))))Y(((Č>-((čOĎ)Oď))Y(Ĝ>-((ĝOĞ)Oğ)))Y(Ĭ>-((ĭOĮ)Oį))))Y(((ā>-((ĀOĂ)Oă))Y(đ>-((ĐOĒ)Oē)))Y(ġ>-((ĠOĢ)Oģ))))Y(((ą>-((ĄOĆ)Oć))Y(ĕ>-((ĔOĖ)Oė)))Y(ĥ>-((ĤOĦ)Oħ))))Y(((ĉ>-((ĈOĊ)Oċ))Y(ę>-((ĘOĚ)Oě)))Y(ĩ>-((ĨOĪ)Oī))))Y(((č>-((ČOĎ)Oď))Y(ĝ>-((ĜOĞ)Oğ)))Y(ĭ>-((ĬOĮ)Oį))))Y(((Ă>-((ĀOā)Oă))Y(Ē>-((ĐOđ)Oē)))Y(Ģ>-((ĠOġ)Oģ))))Y(((Ć>-((ĄOą)Oć))Y(Ė>-((ĔOĕ)Oė)))Y(Ħ>-((ĤOĥ)Oħ))))Y(((Ċ>-((ĈOĉ)Oċ))Y(Ě>-((ĘOę)Oě)))Y(Ī>-((ĨOĩ)Oī))))Y(((Ď>-((ČOč)Oď))Y(Ğ>-((ĜOĝ)Oğ)))Y(Į>-((ĬOĭ)Oį))))Y(((ă>-((ĀOā)OĂ))Y(ē>-((ĐOđ)OĒ)))Y(ģ>-((ĠOġ)OĢ))))Y(((ć>-((ĄOą)OĆ))Y(ė>-((ĔOĕ)OĖ)))Y(ħ>-((ĤOĥ)OĦ))))Y(((ċ>-((ĈOĉ)OĊ))Y(ě>-((ĘOę)OĚ)))Y(ī>-((ĨOĩ)OĪ))))Y(((ď>-((ČOč)OĎ))Y(ğ>-((ĜOĝ)OĞ)))Y(į>-((ĬOĭ)OĮ))))


In [23]:
# Todas las partes individuales de la fórmula completa
partes_formula = [
    # Primer nivel - Conjunciones principales
    '(Ā>-(((āOĂ)Oă)))',
    '(Đ>-(((đOĒ)Oē)))', 
    '(Ġ>-(((ġOĢ)Oģ)))',
    '(Ą>-(((ąOĆ)Oć)))',
    '(Ĕ>-(((ĕOĖ)Oė)))',
    '(Ĥ>-(((ĥOĦ)Oħ)))',
    '(Ĉ>-(((ĉOĊ)Oċ)))',
    '(Ę>-(((ęOĚ)Oě)))',
    '(Ĩ>-(((ĩOĪ)Oī)))',
    '(Č>-(((čOĎ)Oď)))',
    '(Ĝ>-(((ĝOĞ)Oğ)))',
    '(Ĭ>-(((ĭOĮ)Oį)))',
    '(ā>-(((ĀOĂ)Oă)))',
    '(đ>-(((ĐOĒ)Oē)))',
    '(ġ>-(((ĠOĢ)Oģ)))',
    '(ą>-(((ĄOĆ)Oć)))',
    '(ĕ>-(((ĔOĖ)Oė)))',
    '(ĥ>-(((ĤOĦ)Oħ)))',
    '(ĉ>-(((ĈOĊ)Oċ)))',
    '(ę>-(((ĘOĚ)Oě)))',
    '(ĩ>-(((ĨOĪ)Oī)))',
    '(č>-(((ČOĎ)Oď)))',
    '(ĝ>-(((ĜOĞ)Oğ)))',
    '(ĭ>-(((ĬOĮ)Oį)))',
    '(Ă>-(((ĀOā)Oă)))',
    '(Ē>-(((ĐOđ)Oē)))',
    '(Ģ>-(((ĠOġ)Oģ)))',
    '(Ć>-(((ĄOą)Oć)))',
    '(Ė>-(((ĔOĕ)Oė)))',
    '(Ħ>-(((ĤOĥ)Oħ)))',
    '(Ċ>-(((ĈOĉ)Oċ)))',
    '(Ě>-(((ĘOę)Oě)))',
    '(Ī>-(((ĨOĩ)Oī)))',
    '(Ď>-(((ČOč)Oď)))',
    '(Ğ>-(((ĜOĝ)Oğ)))',
    '(Į>-(((ĬOĭ)Oį)))',
    '(ă>-(((ĀOā)OĂ)))',
    '(ē>-(((ĐOđ)OĒ)))',
    '(ģ>-(((ĠOġ)OĢ)))',
    '(ć>-(((ĄOą)OĆ)))',
    '(ė>-(((ĔOĕ)OĖ)))',
    '(ħ>-(((ĤOĥ)OĦ)))',
    '(ċ>-(((ĈOĉ)OĊ)))',
    '(ě>-(((ĘOę)OĚ)))',
    '(ī>-(((ĨOĩ)OĪ)))',
    '(ď>-(((ČOč)OĎ)))',
    '(ğ>-(((ĜOĝ)OĞ)))',
    '(į>-(((ĬOĭ)OĮ)))'
]

print(f"Total de partes individuales: {len(partes_formula)}")

Total de partes individuales: 48


In [24]:
# Diagnóstico de la expansión progresiva
def diagnosticar_expansion_progresiva():
    print("=== DIAGNÓSTICO DE EXPANSIÓN PROGRESIVA ===\n")
    
    # Construir la fórmula progresivamente
    niveles = []
    partes_actuales = []
    
    for i, parte in enumerate(partes_formula, 1):
        partes_actuales.append(parte)
        
        # Construir fórmula progresiva
        if len(partes_actuales) == 1:
            formula_actual = partes_actuales[0]
        else:
            formula_actual = f"({niveles[-1]}Y{parte})"
        
        try:
            print(f"Probando nivel {i}: {len(partes_actuales)} partes")
            print(f"Fórmula (primeros 100 chars): {formula_actual[:100]}...")
            
            # Crear árbol
            A = inorder_to_tree(formula_actual)
            print(f"  ✓ Árbol creado")
            
            # Crear nodo tableaux
            n = nodos_tableaux([A])
            print(f"  ✓ Nodo creado - Alfas: {len(n.alfas)}, Betas: {len(n.betas)}, Literales: {len(n.literales)}")
            
            # Intentar expandir
            hijos = n.expandir()
            print(f"  ✓ Expansión exitosa - {len(hijos)} hijos")
            
            niveles.append(formula_actual)
            print()
            
        except Exception as e:
            print(f"  ✗ ERROR en nivel {i}: {e}")
            print(f"  Tipo error: {type(e).__name__}")
            
            # Información de debug específica para el error de clasificación
            if "'NoneType' object is not subscriptable" in str(e):
                print("  ¡ERROR DE CLASIFICACIÓN ENCONTRADO!")
                print("  Esto indica que alguna subfórmula está devolviendo None en clasificación")
                
                # Diagnosticar qué subfórmula específica está causando el problema
                diagnosticar_subformula_problematica(formula_actual)
            
            return i, formula_actual
    
    print("✓ Todas las expansiones fueron exitosas")
    return None, None

def diagnosticar_subformula_problematica(formula_completa):
    """Encuentra exactamente qué subfórmula está causando el problema"""
    print("\n=== BUSCANDO SUBFÓRMULA PROBLEMÁTICA ===\n")
    
    try:
        A = inorder_to_tree(formula_completa)
        n = nodos_tableaux([A])
        
        print("Analizando fórmulas en el nodo:")
        print(f"Alfas: {len(n.alfas)}")
        for i, (f, s, num, tipo) in enumerate(n.alfas):
            print(f"  Alfa {i}: {s[:50]}...")
            try:
                # Verificar clasificación de cada alfa
                clasif = f.clasifica_para_tableaux()
                print(f"    Clasificación: {clasif}")
            except Exception as e:
                print(f"    ✗ ERROR en clasificación: {e}")
                return f"Alfa problemática: {s}"
        
        print(f"Betas: {len(n.betas)}")
        for i, (f, s, num, tipo) in enumerate(n.betas):
            print(f"  Beta {i}: {s[:50]}...")
            try:
                # Verificar clasificación de cada beta
                clasif = f.clasifica_para_tableaux()
                print(f"    Clasificación: {clasif}")
            except Exception as e:
                print(f"    ✗ ERROR en clasificación: {e}")
                return f"Beta problemática: {s}"
        
        print(f"Literales: {len(n.literales)}")
        for i, (f, s, num, tipo) in enumerate(n.literales):
            print(f"  Literal {i}: {s}")
        
    except Exception as e:
        print(f"Error en diagnóstico: {e}")
    
    return "No se pudo identificar la subfórmula problemática"

# Ejecutar diagnóstico de expansión
nivel_fallido, formula_fallida = diagnosticar_expansion_progresiva()

=== DIAGNÓSTICO DE EXPANSIÓN PROGRESIVA ===

Probando nivel 1: 1 partes
Fórmula (primeros 100 chars): (Ā>-(((āOĂ)Oă)))...
  ✓ Árbol creado
  ✓ Nodo creado - Alfas: 0, Betas: 1, Literales: 0
  ✗ ERROR en nivel 1: 'NoneType' object is not subscriptable
  Tipo error: TypeError
  ¡ERROR DE CLASIFICACIÓN ENCONTRADO!
  Esto indica que alguna subfórmula está devolviendo None en clasificación

=== BUSCANDO SUBFÓRMULA PROBLEMÁTICA ===

Analizando fórmulas en el nodo:
Alfas: 0
Betas: 1
  Beta 0: (Ā>-None)...
    Clasificación: (3, 'beta')
Literales: 0


In [25]:
# CORRECCIÓN: Función expandir completamente revisada
def expandir_corregido(self):
    '''Versión corregida del método expandir'''
    f_alfas = deepcopy(self.alfas)
    f_betas = deepcopy(self.betas)
    f_literales = deepcopy(self.literales)
    
    if len(self.alfas) > 0:
        f, s, num_regla, cl = f_alfas.pop(0)
        print(f"Expandiendo alfa {num_regla}: {s}")
        
        if num_regla == 1:
            formulas = [f.subf.subf]
        elif num_regla == 2:
            formulas = [f.left, f.right]
        elif num_regla == 3:
            formulas = [Negacion(f.subf.left), Negacion(f.subf.right)]
        elif num_regla == 4:
            formulas = [f.subf.left, Negacion(f.subf.right)]
        
        for nueva_f in formulas:
            self._agregar_formula_corregida(nueva_f, f_alfas, f_betas, f_literales)
        
        nuevo_nodo = nodos_tableaux([])
        nuevo_nodo.alfas = f_alfas
        nuevo_nodo.betas = f_betas
        nuevo_nodo.literales = f_literales
        return [nuevo_nodo]
        
    elif len(self.betas) > 0:
        f, s, num_regla, cl = f_betas.pop(0)
        print(f"Expandiendo beta {num_regla}: {s}")
        
        if num_regla == 1:
            B1 = Negacion(f.subf.left)
            B2 = Negacion(f.subf.right)
        elif num_regla == 2:
            B1 = f.left
            B2 = f.right
        elif num_regla == 3:
            B1 = Negacion(f.left)
            B2 = f.right
        
        print(f"B1: {B1} - Tipo: {type(B1)}")
        print(f"B2: {B2} - Tipo: {type(B2)}")
        
        # Crear copias para los dos nodos hijos
        f_alfas1 = deepcopy(f_alfas)
        f_betas1 = deepcopy(f_betas)
        f_literales1 = deepcopy(f_literales)
        
        f_alfas2 = deepcopy(f_alfas)
        f_betas2 = deepcopy(f_betas)
        f_literales2 = deepcopy(f_literales)
        
        # Agregar B1 al primer nodo
        self._agregar_formula_corregida(B1, f_alfas1, f_betas1, f_literales1)
        # Agregar B2 al segundo nodo
        self._agregar_formula_corregida(B2, f_alfas2, f_betas2, f_literales2)
        
        n1 = nodos_tableaux([])
        n1.alfas = f_alfas1
        n1.betas = f_betas1
        n1.literales = f_literales1
        
        n2 = nodos_tableaux([])
        n2.alfas = f_alfas2
        n2.betas = f_betas2
        n2.literales = f_literales2
        
        return [n1, n2]
    else:
        return []

def _agregar_formula_corregida(self, formula, lista_alfas, lista_betas, lista_literales):
    '''Método auxiliar para agregar fórmulas de manera segura'''
    try:
        clasf = formula.clasifica_para_tableaux()
        if clasf[1] == 'alfa':
            lista = lista_alfas
        elif clasf[1] == 'beta':
            lista = lista_betas
        elif clasf[1] == 'literal':
            lista = lista_literales
        else:
            # Si no se puede clasificar, tratarlo como literal por seguridad
            print(f"ADVERTENCIA: Fórmula no clasificada '{formula}', tratando como literal")
            lista = lista_literales
            clasf = (None, 'literal')
        
        # Verificar que no esté duplicada
        strs = [c[1] for c in lista]
        if str(formula) not in strs:
            lista.append((formula, str(formula), *clasf))
            
    except Exception as e:
        print(f"ERROR al clasificar fórmula '{formula}': {e}")
        # En caso de error, agregar como literal
        lista_literales.append((formula, str(formula), None, 'literal'))

# Parche temporal para usar la versión corregida
nodos_tableaux.expandir = expandir_corregido
nodos_tableaux._agregar_formula_corregida = _agregar_formula_corregida

In [29]:
# DIAGNÓSTICO DETALLADO de inorder_to_tree
def diagnosticar_inorder_to_tree():
    cadena = '(Ā>-((āOĂ)Oă))'
    print(f"Analizando cadena: {cadena}")
    print(f"Longitud: {len(cadena)}")
    
    # Simular lo que hace inorder_to_tree
    assert(cadena[0] == "(" and cadena[-1] == ")")
    
    counter = 0
    for i in range(1, len(cadena)):
        if cadena[i] == "(":
            counter += 1
        elif cadena[i] == ")":
            counter -= 1
        elif cadena[i] in CONECTIVOS_BINARIOS and counter == 0:
            print(f"Conectivo principal encontrado en posición {i}: '{cadena[i]}'")
            print(f"Subfórmula izquierda (1:{i}): '{cadena[1:i]}'")
            print(f"Subfórmula derecha ({i+1}:{-1}): '{cadena[i + 1:-1]}'")
            
            # Procesar subfórmulas
            try:
                left_tree = inorder_to_tree(cadena[1:i])
                print(f"Left tree: {left_tree}")
            except Exception as e:
                print(f"Error en left: {e}")
                
            try:
                right_tree = inorder_to_tree(cadena[i + 1:len(cadena)-1])
                print(f"Right tree: {right_tree}")
            except Exception as e:
                print(f"Error en right: {e}")
            
            break

diagnosticar_inorder_to_tree()

Analizando cadena: (Ā>-((āOĂ)Oă))
Longitud: 14
Conectivo principal encontrado en posición 2: '>'
Subfórmula izquierda (1:2): 'Ā'
Subfórmula derecha (3:-1): '-((āOĂ)Oă)'
Left tree: Ā
Right tree: -((āOĂ)Oă)


In [27]:
forms = [final]
forms = [inorder_to_tree(f) for f in forms]
n = nodos_tableaux(forms)
I = primero_profundidad(n)
print("Resultado:", I)

Expandiendo alfa 2: ((((((((((((((((((Ā>-((āOĂ)Oă))Y(Đ>-((đOĒ)Oē)))Y(Ġ>-((ġOĢ)Oģ)))Y(((Ą>-((ąOĆ)Oć))Y(Ĕ>-((ĕOĖ)Oė)))Y(Ĥ>-((ĥOĦ)Oħ))))Y(((Ĉ>-((ĉOĊ)Oċ))Y(Ę>-((ęOĚ)Oě)))Y(Ĩ>-((ĩOĪ)Oī))))Y(((Č>-((čOĎ)Oď))Y(Ĝ>-((ĝOĞ)Oğ)))Y(Ĭ>-((ĭOĮ)Oį))))Y(((ā>-((ĀOĂ)Oă))Y(đ>-((ĐOĒ)Oē)))Y(ġ>-((ĠOĢ)Oģ))))Y(((ą>-((ĄOĆ)Oć))Y(ĕ>-((ĔOĖ)Oė)))Y(ĥ>-((ĤOĦ)Oħ))))Y(((ĉ>-((ĈOĊ)Oċ))Y(ę>-((ĘOĚ)Oě)))Y(ĩ>-((ĨOĪ)Oī))))Y(((č>-((ČOĎ)Oď))Y(ĝ>-((ĜOĞ)Oğ)))Y(ĭ>-((ĬOĮ)Oį))))Y(((Ă>-((ĀOā)Oă))Y(Ē>-((ĐOđ)Oē)))Y(Ģ>-((ĠOġ)Oģ))))Y(((Ć>-((ĄOą)Oć))Y(Ė>-((ĔOĕ)Oė)))Y(Ħ>-((ĤOĥ)Oħ))))Y(((Ċ>-((ĈOĉ)Oċ))Y(Ě>-((ĘOę)Oě)))Y(Ī>-((ĨOĩ)Oī))))Y(((Ď>-((ČOč)Oď))Y(Ğ>-((ĜOĝ)Oğ)))Y(Į>-((ĬOĭ)Oį))))Y(((ă>-((ĀOā)OĂ))Y(ē>-((ĐOđ)OĒ)))Y(ģ>-((ĠOġ)OĢ))))Y(((ć>-((ĄOą)OĆ))Y(ė>-((ĔOĕ)OĖ)))Y(ħ>-((ĤOĥ)OĦ))))Y(((ċ>-((ĈOĉ)OĊ))Y(ě>-((ĘOę)OĚ)))Y(ī>-((ĨOĩ)OĪ))))Y(((ď>-((ČOč)OĎ))Y(ğ>-((ĜOĝ)OĞ)))Y(į>-((ĬOĭ)OĮ))))
Expandiendo alfa 2: (((((((((((((((((Ā>-((āOĂ)Oă))Y(Đ>-((đOĒ)Oē)))Y(Ġ>-((ġOĢ)Oģ)))Y(((Ą>-((ąOĆ)Oć))Y(Ĕ>-((ĕOĖ)Oė)))Y(Ĥ>-((ĥOĦ)Oħ))))Y(((Ĉ>-((ĉOĊ)Oċ))Y(Ę>-((ęOĚ)Oě)))Y

---

# Sección opcional: Eficiencia de los algoritmos <a class="anchor" id="efi"></a>


([Volver al inicio](#inicio))

Vamos a verificar empíricamente cuál función es más rápida para expandir el árbol del tableaux de una fórmula dada. Para ello, crearemos fórmulas más y más grandes para probar en ellas nuestros tres algoritmos.

Definimos nuestras funciones para medir tiempos en una lista de entradas: 

In [28]:
from time import time
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

def obtiene_tiempos(fun, args, num_it=100):
    tiempos_fun = []
    for i in range(num_it):
        arranca = time()
        x = fun(*args)
        para = time()
        tiempos_fun.append(para - arranca)
    return tiempos_fun

def compara_entradas_funs(funs, nombres_funs, lista_args, N=100):
    entradas = []
    funcion = []
    tiempos = []
    lista_dfs = []
    for i, args in enumerate(lista_args):
        for j, fun in enumerate(funs):
            t = obtiene_tiempos(fun, [args], N)
            tiempos += t
            n = len(t)
            entradas += [i+1]*n
            funcion += [nombres_funs[j]]*n
        df = pd.DataFrame({'Long_entrada':entradas, 
                           'Funcion':funcion,
                           'Tiempo_prom':tiempos})
        lista_dfs.append(df)
    df = pd.concat(lista_dfs).reset_index()
    sns.lineplot(x='Long_entrada',y='Tiempo_prom',hue='Funcion',data=df)
    plt.show()

ModuleNotFoundError: No module named 'seaborn'

Creamos nuestra lista de fórmulas:

In [ ]:
from random import uniform, choice

cantidad = 20
letras = [chr(i) for i in range(256, 256+cantidad)]
lista = [letras[0]]
formula = letras[0]
for p in letras[1:]:
    neg1 = '-' if uniform(0,1) > .5 else ''
    neg2 = '-' if uniform(0,1) > .5 else ''
    conectivo = choice(['Y','O','>'])
    formula = neg2 + "(" + formula + conectivo + neg1 + p + ")"
    lista.append(formula)

#lista

Y, finalmente, medimos los tiempos de ejecución para cada fórmula:

In [ ]:
nombres = ['primero anchura', 'primero profundidad', 'backtracking']
anchura = lambda formula: primero_anchura(nodos_tableaux([inorder_to_tree(formula)]))
profundidad = lambda formula: primero_profundidad(nodos_tableaux([inorder_to_tree(formula)]))
backtrack = lambda formula: backtracking(nodos_tableaux([inorder_to_tree(formula)]))
funs = [anchura, profundidad, backtrack]
compara_entradas_funs(funs, nombres, lista)

En esta gráfica es muy fácil apreciar que, a medida que aumentamos el número de letras proposicionales distintas en la fórmula de entrada, los tres algoritmos se tardan más en determinar si ella es satisfacible. De los tres algoritmos, el peor es el de backtracking y el mejor es el de primero en profundidad. No obstante, vemos que el número de letras proposicionales no es el único factor que influye en el tiempo de ejecución. También influye la estructura de la fórmula. Más adelante estudiaremos el comportamiento de los algoritmos de acuerdo a distintos tipos de fórmulas.

---